# NASA Throughput Prediction Challenge
### Looking at the data!
- http://bitgrit.net/competition/23

Problem statement: "Given an airport ABC, and a timestamp T, what is the expected number of arriving flights (throughput) in the next 3 hours with a resolution of 15-minutes time buckets."

### Input Data
- Two classifcations of data flight info, weather. 
- 4 types of data: FUSER, METAR, TAF, CWAM 

What are the charactersistics and differences? 



FUSER: 
- 8 types of CSV
- {airport}_{date_range}.{file_type}_data_set.csv

```
configs_data_set (D-ATIS Data):
Columns: airport_id, data_header, src_addr, datis_time (time of D-ATIS message), start_time (time configuration starts), weather_report, departure_runways (parsed departure runways), arrival_runways (parsed arrival runways), timestamp_source_received, timestamp_source_processed, invalid_departure_runways, invalid_arrival_runways, departure_runway_string, arrival_runway_string, airport_configuration_name.
Purpose: Contains airport configuration and weather details extracted from D-ATIS messages, used for understanding airport configurations.


runways_data_set (Arrival/Departure Detection):
Columns: gufi (unique flight identifier), arrival_runway_actual_time (actual time of arrival), arrival_runway_actual (runway used for arrival), departure_runway_actual_time (actual time of departure), departure_runway_actual (runway used for departure).
Purpose: Provides actual times and runways for arrivals and departures. Primary Source for Target Variables: Used to count the number of arrivals in 15-minute intervals for the next 3 hours (target variable).

first_position_data_set:
Columns: gufi (unique flight identifier), time_first_tracked (time when the flight is first detected).
Purpose: Tracks the initial detection time for each flight.

TBFM_data_set (Time-Based Flow Management Data):
Columns: gufi, timestamp (time estimate is made), arrival_runway_sta (scheduled time of arrival).
Purpose: Contains scheduled times of arrival for flights.

TFM_track_data_set (Traffic Flow Management Data):
Columns: gufi, timestamp (time estimate is made), arrival_runway_estimated_time (estimated time of arrival).
Purpose: Provides estimated arrival times for flights; helps to predict future arrival times.

ETD_data_set (Estimated Time of Departure):
Columns: gufi, timestamp (time estimate is made), departure_runway_estimated_time (estimated departure time).
Purpose: Provides estimated departure times for flights.

LAMP_data_set (Local Aviation MOS Program Data):
Columns: timestamp (time of forecast), forecast_timestamp (forecasted time), temperature (Fahrenheit), wind_direction (0-36 scale), wind_speed (knots), wind_gust (knots), cloud_ceiling (code from 1 to 8), visibility (code from 1 to 7), cloud (categories like CL, FW, SC, BK, OV), lightning_prob (N, L, M, H), precip (True/False).
Purpose: Provides short-term weather forecasts relevant for airport operations.

MFS_data_set (FAA SWIM Feeds):
Columns: gufi, aircraft_engine_class (e.g., JET), aircraft_type (e.g., Boeing 737), arrival_aerodrome_icao_name (arrival airport ICAO code), major_carrier (e.g., UAL for United Airlines), flight_type (e.g., SCHEDULED_AIR_TRANSPORT), isarrival (True/False), isdeparture (True/False), arrival_stand_actual (gate at arrival), arrival_stand_actual_time (time at arrival gate), arrival_runway_actual (actual arrival runway), arrival_runway_actual_time (actual arrival time), departure_stand_actual (gate at departure), departure_stand_actual_time (time of push-back), departure_runway_actual (actual departure runway), departure_runway_actual_time (actual departure time).
Purpose: Detailed flight information from FAA SWIM feeds, used for understanding flight paths, types, and status.
```

The set: runways_data_set (Arrival/Departure Detection) an important dataset, it allows us to count

## METAR
- Provides information about what weather around airports look like
- https://en.wikipedia.org/wiki/METAR


Example format: 

```
2022/09/25 00:00
KSEA 250000Z 05010KT 9999 SCT020 FEW021CB 31/25 Q1011

2022/09/25 00:00: Date and time of the observation.
KSEA: Station identifier (e.g., Seattle International Airport).
250000Z: Day of the month and time of the report in UTC (e.g., 25th day at 00:00 UTC).
05010KT: Wind direction (050 degrees) and speed (10 knots).
9999: Visibility in meters (e.g., 9999 meters or 10 kilometers).
SCT020 FEW021CB: Cloud cover information (scattered clouds at 2,000 feet and few cumulonimbus clouds at 2,100 feet).
31/25: Temperature (31°C) and dew point (25°C).
Q1011: Altimeter setting (e.g., 1011 hPa).
```


# TAF
- Prediction data used by aviators 
- Data predicts the future weather predictions from 24 - 30 hrs. Updated every 6 hours.
- https://en.wikipedia.org/wiki/Terminal_aerodrome_forecast

```
2022/09/24 00:00

 
TAF KJFK 242200Z 2500/2524 07005KT 9999 SCT025 TX34/2517Z TN23/2508Z PROB30
          TEMPO 2508/2511 VRB01KT 3000BR SCT005
          TEMPO 2517/2522 5000 TSRA FEW018CB SCT020
2022/09/24 00:00: Date and time the forecast was issued.
TAF KJFK: The report type and station identifier (e.g., John F. Kennedy International Airport).
242200Z: Day of the month and time of issuance in UTC (e.g., 24th day at 22:00 UTC).
2500/2524: Valid period of the forecast (e.g., from 00:00 UTC on the 25th to 24:00 UTC on the 25th).
07005KT: Forecasted wind direction (070 degrees) and speed (5 knots).
9999: Forecasted visibility (e.g., 9999 meters or 10 kilometers).
SCT025: Scattered clouds at 2,500 feet.
TX34/2517Z: Maximum temperature of 34°C expected at 17:00 UTC on the 25th.
TN23/2508Z: Minimum temperature of 23°C expected at 08:00 UTC on the 25th.
PROB30 TEMPO: 30% probability of temporary conditions, followed by the specific forecast.
```


## CWAM
- Convective Weather Avoidance Model
- This data is a forecast on the how convective forces may impact by convective weather
- Dataset provides polygons for a given altitude 
- Forcasts provided every 15 minutes, forecasts made up to 2hrs in the future with a 5 minute interval. 

- Name of files: 
```
<YYYY>_<MM>_<DD>_<HH>_<MM>_GMT.Forecast.h5.CWAM.h5.bz2
Example: 2022_12_18_23_45_GMT.Forecast.h5.CWAM.h5.bz2 
```

```
Keys Structure: Deviation Probability/FCST<Forecast Time>/FLVL<Flight Level>/Contour/TRSH<Threshold>/POLY<Polygon Number>
Forecast Time (FCST): Represents the forecast minute offset (e.g., FCST000 for the initial forecast : time of the file + 0 minute, FCST010 would be time of the file + 10 minute, i.e. 10 minutes in the future).
Flight Level (FLVL): Represents the altitude of the forecast in flight levels (e.g., FLVL250 for 25,000 feet).
Threshold (TRSH): Represents the deviation probability threshold (e.g., TRSH060 for greater than 60%).
Polygon Number (POLY): Represents the polygon index number

```

In [1]:
## I'm kind of confused about the train test split. Why is it 24 days and then 8 days? 

## Evaluation Metric

"The evaluation metric is based on Root Mean Squared Error (RMSE). Specifically, it is calculated as exp(-RMSE/K), where K is a normalization factor of 10, and exp is the exponential function in order for it to be a number between 0.0 and 1.0, with 1.0 representing a perfect score (RMSE = 0)."


### Rules 
- We are allowed to build models that take other datasets not provided, so long as those datasets are in the public domain. Additionally, we may use pre-trained models. It is very important that we do not make predictions using information that isn't available at the time of inference. Keep this in mind when building the model

### Thoughts so far. 

Goal -: Create the model below: 

Model(time: TIMESTAMP, airport: AIRPORT_IDENTIFIER) -> arrivals the next 3 hours in 15 minutes buckets.

We are provided with data about the actual training time, but do we have the real resutls 

Questions: 
- What are the differences between test and train data? 
- What is a baseline using basic tools can we implement? 
- What is the structure of the test data? 
- Why is this even an important problem worth solving? Predicting buckets of airplane arrivals for the next 3 hours doesn't seem all too useful. I mean the capital requirements don't change much. Airports can really change much stuff with this can they? 

        - Maybe they can! Airports are expensive to run, so overstaffing 24/7 may be too expensive, but understaffing could be costly. Hence if a 3 hour projection starts to show an arrival amount that exceeds current labor available, the airport could call up labor on standby. However, it still seems weird since 3 hours isn't much runway. 
        

### A Reformulation of the problem:

Let's reduce this problem to one of a simpler domain to understand casual relationships. Imagine there are cab drivers who want to arrive to different hotels. Cab drivers can be picked up stochastically, but there are also many fixed appointments that existed before hand. Given a bunch of hotels, and times we want to predict the amount of cabs that will arrive to a hotel for a time interval, separated into time buckets. 

What influences the cab driver's behaviors? What are the causal relationships? 

Lets make this problem a little bit weaker. Let's assume cab's cant be hauled stochastically. They are planned in advanced and all riders respect the appointments. So we can formally say we have a list of times and planned movements of cabs. 

Cabs must travel though a space. Lets represent that space as a weighted graph G = (V,E) where some edge e in E corresponds to a road in the real world, and v represents connections between two roads. This edge is also weighted, by some function or value. **Key Point: A function seems like a better idea for weights, this comes from the idea of [KANs](https://arxiv.org/abs/2404.19756).** 

Regardless this Graph is dynamically changing. We can also add another constaint where  sometimes edges may disappear. Doing this runs us back into the field of computer networking and the Bellman-Fold Equation

Let us also assume that 